In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Encoder(nn.Module):
    def __init__(self, time_size, hidden_size, feature_size):
        super(Encoder, self).__init__()
        self.time_size = time_size
        self.hidden_size = hidden_size
        self.feature_size = feature_size

        self.lstm = nn.LSTM(input_size=feature_size, 
                             hidden_size=hidden_size)

    def forward(self, xs, hs):
        outputs, (hidden, cell) = self.lstm(xs, hs)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size, feature_size):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.feature_size = feature_size
        self.output_size = output_size

        self.lstm = nn.LSTM(input_size=feature_size,
                            hidden_size = hidden_size)
        self.fc1 = nn.Linear(hidden_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, xs, hidden, cell):
        '''
        xs : batch, time, feture
        '''
        output, (hidden, cell) = self.lstm(xs, (hidden, cell))
        output = self.fc1(output.squeeze(0))
        output = self.fc2(output)

        return output, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, xs, t):
        hidden, cell = self.encoder(xs)
        